# Pouring Food OWLReady Integration Notebook
OWLReady Integration with owlready2 for the Food Cutting SPARQL Queries from: https://github.com/Food-Ninja/PouringFood/blob/main/jupyter/PouringFoodQueries.ipynb

In [ ]:
from owlready2 import get_ontology, Restriction, owlready, Or
from parso.python.tree import Class

- The URL can be a file from the internet but also a local file
- Defining the Namespaces globally so they can be accessed by every function
- Accessing a class requires to call the Namespace for example, the class "apple" (IRI: "http://purl.obolibrary.org/obo/FOODON_03301710") corresponds to OBO.FOODON_03301710, as the OBO namespace is defined as such.

In [ ]:
url = "https://raw.githubusercontent.com/Food-Ninja/PouringFood/main/owl/PouringFood.owl"
onto = get_ontology(url).load()
SOMA = onto.get_namespace("http://www.ease-crc.org/ont/SOMA.owl#")
POUR = onto.get_namespace("http://www.ease-crc.org/ont/pouring_food#")
DUL = onto.get_namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
OBO = onto.get_namespace("http://purl.obolibrary.org/obo/")
QUDT = onto.get_namespace("http://qudt.org/schema/qudt#")


# What Tool should be used? -Query
- Returns the required tool to execute the task. Requires a task as input

In [ ]:
def whattool(task):
    for parent in task.is_a:
        if isinstance(parent, Restriction) and parent.property == DUL.hasParticipant:
            return parent.value

# What is the consistency of the Food Object? -Query
- Returns the Consistency (or multiple Consistencies) of the input Food Object.
- Some Objects inherit their Consistency attribute from their superclass
- Some Objects can have more then one consistency (situational). In this case the Consistencies are united with the logical "Or" operator

In [ ]:
def get_consistency(food_cls):
    for parent in food_cls.is_a:
        if isinstance(parent, Restriction):
            if parent.property == POUR.hasConsistency:
                return parent.value
        
    for parent in food_cls.is_a:
        for child in parent.is_a:
            if isinstance(child, Restriction):
                if child.property == POUR.hasConsistency:
                    return child.value

# Which Angle class corresponds to the given Consistency? -Query
- Returns the Angle class that corresponds to the input consistency.
- The input Consistency can be also queried with the "get_consistency" function while giving a Food Object as input.
- We iterate over the Subclasses of the PouringAngle Class and check for the Pour.hasInputObject - Restrictions.
- This Restriction can have only one InputObject (Pour.Liquid) or multiple (every other Consistency)
- We check for the subclass in which the consistency is represented and return this particular subclass

In [ ]:
def get_angle_class_from_consistency(consistency):
    consistencies = []
    for subclass in POUR.PouringAngle.subclasses():
        for parent in subclass.is_a:
            if not isinstance(parent, Restriction):
               continue
            if parent.property == POUR.hasInputObject:
                for child in parent.value.Classes:                
                    if isinstance(child, Restriction):
                        if isinstance(child.value, Or):
                            for i in child.value.Classes:
                                consistencies.append(i)
                        consistencies.append(child.value)
                        if consistency in consistencies:
                            consistencies.clear()
                            return subclass
                    

# Which Duration class corresponds to the given Consistency? -Query
- Returns the Duration class that corresponds to the input consistency.
- The input Consistency can be also queried with the "get_consistency" function while giving a Food Object as input.
- We iterate over the Subclasses of the Duration Class and check for the Pour.hasInputObject - Restrictions.
- This Restriction can have only one InputObject (Pour.Liquid) or multiple (every other Consistency)
- We check for the subclass in which the consistency is represented and return this particular subclass

In [ ]:
def get_duration_class_from_consistency(consistency):
    consistencies = []
    for subclass in POUR.PouringDuration.subclasses():
        for parent in subclass.is_a:
            if not isinstance(parent, Restriction):
               continue
            if parent.property == POUR.hasInputObject:
                for child in parent.value.Classes:                
                    if isinstance(child, Restriction):
                        if isinstance(child.value, Or):
                            for i in child.value.Classes:
                                consistencies.append(i)
                        consistencies.append(child.value)
                        if consistency in consistencies:
                            consistencies.clear()
                            return subclass



# Get the Angles required for executing the task.
- Returning the minimal and maximal angles values.
- There are 3 cases that can occur, regarding the consistency of the food object:
1. The Food object class can have multiple consistencies defined, if one of them is the Liquid class, it is required to consider both PouringAngle classes as they differ for Liquid consistency and any other consistency.
2. The Food object class can have multiple consistencies defined, if none of them is the Liquid class, we choose the first defined consistency, because every other consistency then Liquid returns the same PouringAngle values.
3. The Food object class has only one consistency defined, and returns the associated PouringAngle values.
- The min and max values for the PouringAngle are first appended to a list, these values are then assigned with the min() and max()- functions, and returned

In [ ]:
def get_angles(food_cls):
    angles = []
    if isinstance(get_consistency(food_cls), Or):
        
        if POUR.Liquid in get_consistency(food_cls).Classes:
            for parent in get_angle_class_from_consistency(POUR.Liquid).is_a:
                if isinstance(parent, Restriction):
                    if parent.property == QUDT.valueQuantity:
                        angles.append(parent.cardinality)
            
            for parent in get_angle_class_from_consistency(POUR.Moist).is_a:
                if isinstance(parent, Restriction):
                    if parent.property == QUDT.valueQuantity:
                        angles.append(parent.cardinality)
    
        if POUR.Liquid not in get_consistency(food_cls).Classes:
                for parent in get_angle_class_from_consistency(get_consistency(food_cls).Classes[0]).is_a:
                    if isinstance(parent, Restriction):
                        if parent.property == QUDT.valueQuantity:
                            angles.append(parent.cardinality)
    
    else:        
        for parent in get_angle_class_from_consistency(get_consistency(food_cls)).is_a:
            if isinstance(parent, Restriction):
                if parent.property == QUDT.valueQuantity:
                    angles.append(parent.cardinality)
                
                
    
    min_angle = min(angles)
    max_angle = max(angles)
    
    return min_angle, max_angle

# Get the Duration required for executing the task.
- Returning the minimal and maximal duration values.
- There are 3 cases that can occur, regarding the consistency of the food object:
1. The Food object class can have multiple consistencies defined, if one of them is the Liquid class, it is required to consider both Duration classes as they differ for Liquid consistency and any other consistency.
2. The Food object class can have multiple consistencies defined, if none of them is the Liquid class, we choose the first defined consistency, because every other consistency then Liquid returns the same Duration values.
3. The Food object class has only one consistency defined, and returns the associated Duration values.
- The min and max values for the Duration are first appended to a list, these values are then assigned with the min() and max()- functions, and returned

In [ ]:
def get_durations(food_cls):
    durations = []
    
    if isinstance(get_consistency(food_cls), Or):
        if POUR.Liquid in get_consistency(food_cls).Classes:
            
            for parent in get_duration_class_from_consistency(POUR.Liquid).is_a:
                if isinstance(parent, Restriction):
                    if parent.property == QUDT.valueQuantity:
                        durations.append(parent.cardinality)
            
            for parent in get_angle_class_from_consistency(POUR.Moist).is_a:
                if isinstance(parent, Restriction):
                    if parent.property == QUDT.valueQuantity:
                        durations.append(parent.cardinality)
        
        if POUR.Liquid not in get_consistency(food_cls).Classes:
            for parent in get_duration_class_from_consistency(get_consistency(food_cls).Classes[0]).is_a:
                if isinstance(parent, Restriction):
                    if parent.property == QUDT.valueQuantity:
                        durations.append(parent.cardinality)
    else:
        for parent in get_duration_class_from_consistency(get_consistency(food_cls)).is_a:
            if isinstance(parent, Restriction):
                if parent.property == QUDT.valueQuantity:
                    durations.append(parent.cardinality)
    
    min_duration = min(durations)
    max_duration = max(durations)
    
    return min_duration, max_duration

In [ ]:
# Pipeline
task = POUR.Draining
Food_object = POUR.marinade
print("Input Task: " + str(task))
print("Input Food Object: " + str(Food_object))
print("Needed Tool:")
print(whattool(task))
print("Pouring Angles:")
print(get_angles(food_cls=Food_object))
print("Durations:")
print(get_durations(food_cls=Food_object))